In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

DATA WRANGLING

In [47]:
df = pd.read_csv('result_final.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,link,text,title,date,keywords,summary,title_summary
0,0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
2,2,2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho..."
3,3,3,http://techcrunch.com/2020/09/09/groww-an-inve...,Even as more than 150 million people are using...,"Groww, an investment app for millennials in In...",2020-09-09 00:00:00,"['world', 'yc', 'continuity', 'stocks', 'groww...","YC Continuity, the growth-stage investment fun...","Groww, an investment app for millennials in In..."
4,4,4,http://techcrunch.com/2020/09/15/lanzatech-is-...,As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...,2020-09-15 00:00:00,"['production', 'distributed', 'developing', 's...",As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...


In [48]:
df=df.iloc[:,3:]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           2184 non-null   object
 1   title          2187 non-null   object
 2   date           1691 non-null   object
 3   keywords       2187 non-null   object
 4   summary        2184 non-null   object
 5   title_summary  2184 non-null   object
dtypes: object(6)
memory usage: 102.8+ KB


In [49]:
df=df.dropna().reset_index(drop=True)
df['date'] = df['date'].str.split().str[0]
df['date'] = pd.to_datetime(df['date'])
#df['keywords'] = df['keywords'].str.split()

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1689 entries, 0 to 1688
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   text           1689 non-null   object        
 1   title          1689 non-null   object        
 2   date           1689 non-null   datetime64[ns]
 3   keywords       1689 non-null   object        
 4   summary        1689 non-null   object        
 5   title_summary  1689 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 79.3+ KB


In [51]:
df.head()

,text,title,date,keywords,summary,title_summary
0,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...
1,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
2,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho..."
3,Even as more than 150 million people are using...,"Groww, an investment app for millennials in In...",2020-09-09,"['world', 'yc', 'continuity', 'stocks', 'groww...","YC Continuity, the growth-stage investment fun...","Groww, an investment app for millennials in In..."
4,As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...,2020-09-15,"['production', 'distributed', 'developing', 's...",As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...


1. MATRIX FACTORIZATION

In [52]:
df['text_features'] = df['title'] + ' ' + df['text'] + ' ' + df['summary'] + ' ' + df['title_summary'] + ' ' + df['keywords']
vectorizer = TfidfVectorizer()

vectorized_matrix = vectorizer.fit_transform(df['text_features'])

In [53]:
vectorized_matrix

<1689x35412 sparse matrix of type '<class 'numpy.float64'>'
	with 493539 stored elements in Compressed Sparse Row format>

In [54]:
svd = TruncatedSVD(n_components=100)

svd_matrix = svd.fit_transform(vectorized_matrix)

In [60]:
def recommendation_1(article_index):
    article_scores = svd_matrix[article_index]
    article_similarities = svd_matrix.dot(article_scores)

    top_articles_indices = article_similarities.argsort()[::-1][1:6]

    similar_articles = df.loc[top_articles_indices]

    return similar_articles

In [61]:
index = 5
df.iloc[index]

text             More than a third of small and medium-sized bu...
title            Facebook announces $4.3 million grant for smal...
date                                           2020-09-15 00:00:00
keywords         ['businesses', 'pandemic', 'support', 'cash', ...
summary          The social giant today announced a grant of $4...
title_summary    Facebook announces $4.3 million grant for smal...
text_features    Facebook announces $4.3 million grant for smal...
Name: 5, dtype: object

In [62]:
recommendation_1(index)

,text,title,date,keywords,summary,title_summary,text_features
963,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16,"['facebook', 'users', 'startups', 'role', 'inv...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...,Facebook addresses political controversy in In...
967,More than a third of small and medium-sized bu...,Facebook announces $4.3 million grant for smal...,2020-09-15,"['facebook', 'businesses', 'today', 'cards', '...",The social giant today announced a grant of $4...,Facebook announces $4.3 million grant for smal...,Facebook announces $4.3 million grant for smal...
5,More than a third of small and medium-sized bu...,Facebook announces $4.3 million grant for smal...,2020-09-15,"['businesses', 'pandemic', 'support', 'cash', ...",The social giant today announced a grant of $4...,Facebook announces $4.3 million grant for smal...,Facebook announces $4.3 million grant for smal...
794,The Interface is a daily column and newsletter...,What Facebook should do about its Kenosha problem,2020-09-01,"['problem', 'facebook', 'page', 'reports', 'fa...","The afternoon before the murders, a 3,000-memb...",What Facebook should do about its Kenosha prob...,What Facebook should do about its Kenosha prob...
458,The Interface is a daily column and newsletter...,What Facebook should do about its Kenosha problem,2020-09-01,"['content', 'page', 'guard', 'problem', 'kenos...","The afternoon before the murders, a 3,000-memb...",What Facebook should do about its Kenosha prob...,What Facebook should do about its Kenosha prob...


2. CONTENT-BASED FILTERING

In [63]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [95]:
df['text_features'] = df['title'] + ' ' + df['text'] + ' ' + df['summary'] + ' ' + df['title_summary'] + ' ' + df['keywords']

vectorizer = TfidfVectorizer()
vectorized_matrix = vectorizer.fit_transform(df['text_features'])

In [96]:
pairwise_corr_matrix = 1 - pairwise_distances(vectorized_matrix.toarray(), metric='correlation')

In [97]:
def recommendation_2(article_index):
    corr_scores = list(enumerate(pairwise_corr_matrix[article_index]))
    corr_scores = sorted(corr_scores, key=lambda x: x[1], reverse=True)

    similar_corr_scores = corr_scores[1:6]
    similar_article_indices = [i[0] for i in similar_corr_scores]
    return df.loc[similar_article_indices]

In [98]:
df.iloc[index]

text             More than a third of small and medium-sized bu...
title            Facebook announces $4.3 million grant for smal...
date                                           2020-09-15 00:00:00
keywords         ['businesses', 'pandemic', 'support', 'cash', ...
summary          The social giant today announced a grant of $4...
title_summary    Facebook announces $4.3 million grant for smal...
text_features    Facebook announces $4.3 million grant for smal...
Name: 5, dtype: object

In [99]:
recommendation_2(index)

,text,title,date,keywords,summary,title_summary,text_features
967,More than a third of small and medium-sized bu...,Facebook announces $4.3 million grant for smal...,2020-09-15,"['facebook', 'businesses', 'today', 'cards', '...",The social giant today announced a grant of $4...,Facebook announces $4.3 million grant for smal...,Facebook announces $4.3 million grant for smal...
1,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...,Facebook addresses political controversy in In...
963,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16,"['facebook', 'users', 'startups', 'role', 'inv...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...,Facebook addresses political controversy in In...
174,The Interface is a daily column and newsletter...,What Facebook should do about its Kenosha problem,2020-09-01,"['content', 'page', 'guard', 'problem', 'kenos...","The afternoon before the murders, a 3,000-memb...",What Facebook should do about its Kenosha prob...,What Facebook should do about its Kenosha prob...
458,The Interface is a daily column and newsletter...,What Facebook should do about its Kenosha problem,2020-09-01,"['content', 'page', 'guard', 'problem', 'kenos...","The afternoon before the murders, a 3,000-memb...",What Facebook should do about its Kenosha prob...,What Facebook should do about its Kenosha prob...


3. NEAREST NEIGHBORS ALGORITHM

In [100]:
from sklearn.neighbors import NearestNeighbors

In [114]:
df['text_features'] = df['title'] + ' ' + df['text'] + ' ' + df['summary'] + ' ' + df['title_summary'] + ' ' + df['keywords']

vectorizer = TfidfVectorizer()
vectorized_matrix = vectorizer.fit_transform(df['text_features'])
vectorized_matrix = vectorized_matrix.toarray()

In [118]:
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine')

knn_model.fit(vectorized_matrix)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [119]:
def recommendation_3(article_index):
    distances, indices = knn_model.kneighbors([vectorized_matrix[article_index]])
    similar_article_indices = indices[0][1:]
    return df.loc[similar_article_indices]

In [120]:
recommendation_3(index)

,text,title,date,keywords,summary,title_summary,text_features
967,More than a third of small and medium-sized bu...,Facebook announces $4.3 million grant for smal...,2020-09-15,"['facebook', 'businesses', 'today', 'cards', '...",The social giant today announced a grant of $4...,Facebook announces $4.3 million grant for smal...,Facebook announces $4.3 million grant for smal...
963,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16,"['facebook', 'users', 'startups', 'role', 'inv...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...,Facebook addresses political controversy in In...
1,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...,Facebook addresses political controversy in In...
794,The Interface is a daily column and newsletter...,What Facebook should do about its Kenosha problem,2020-09-01,"['problem', 'facebook', 'page', 'reports', 'fa...","The afternoon before the murders, a 3,000-memb...",What Facebook should do about its Kenosha prob...,What Facebook should do about its Kenosha prob...
174,The Interface is a daily column and newsletter...,What Facebook should do about its Kenosha problem,2020-09-01,"['content', 'page', 'guard', 'problem', 'kenos...","The afternoon before the murders, a 3,000-memb...",What Facebook should do about its Kenosha prob...,What Facebook should do about its Kenosha prob...


4. CLUSTERING ALGORITHM

In [126]:
from sklearn.cluster import KMeans

In [127]:
df['text_features'] = df['title'] + ' ' + df['text'] + ' ' + df['summary'] + ' ' + df['title_summary'] + ' ' + df['keywords']

vectorizer = TfidfVectorizer()
vectorized_matrix = vectorizer.fit_transform(df['text_features'])
vectorized_matrix = vectorized_matrix.toarray()

In [128]:
kmeans_model = KMeans(n_clusters = 15, random_state = 37)

kmeans_model.fit(vectorized_matrix)

C:\Users\kulje\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=15, random_state=37)

In [129]:
cluster_labels = kmeans_model.labels_

In [134]:
def recommendation_4(article_index):
    article_cluster_label = cluster_labels[article_index]
    cluster_articles = [df.loc[i] for i, label in enumerate(cluster_labels) if label == article_cluster_label and i != article_index]
    return pd.DataFrame(cluster_articles)

In [136]:
df.loc[index]

text             More than a third of small and medium-sized bu...
title            Facebook announces $4.3 million grant for smal...
date                                           2020-09-15 00:00:00
keywords         ['businesses', 'pandemic', 'support', 'cash', ...
summary          The social giant today announced a grant of $4...
title_summary    Facebook announces $4.3 million grant for smal...
text_features    Facebook announces $4.3 million grant for smal...
Name: 5, dtype: object

In [135]:
recommendation_4(index)

,text,title,date,keywords,summary,title_summary,text_features
1,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...,Facebook addresses political controversy in In...
85,Facebook is planning to expand its dedicated n...,Facebook plans to expand its news tab beyond t...,2020-08-25,"['content', 'video', 'possible', 'expand', 'ta...",Facebook is planning to expand its dedicated n...,Facebook plans to expand its news tab beyond t...,Facebook plans to expand its news tab beyond t...
86,The Interface is a daily column and newsletter...,Facebook’s big QAnon crackdown might have come...,2020-08-20,"['come', 'late', 'conspiracy', 'big', 'theory'...",The “Tide Pods challenge” surged on social net...,Facebook’s big QAnon crackdown might have come...,Facebook’s big QAnon crackdown might have come...
87,"Ankhi Das, a top Facebook executive in India, ...",Top Facebook executive in India files criminal...,2020-08-19,"['journalist', 'das', 'tiwari', 'wsj', 'public...",Several users also tweeted and submitted posts...,Top Facebook executive in India files criminal...,Top Facebook executive in India files criminal...
89,After restricting a group critical of Thailand...,After restricting a group critical of Thailand...,2020-08-24,"['access', 'critical', 'restrict', 'thailand',...",After restricting a group critical of Thailand...,After restricting a group critical of Thailand...,After restricting a group critical of Thailand...
173,The Interface is a daily column and newsletter...,How Facebook’s new election rules sidestep the...,2020-09-04,"['rules', 'sidestep', 'speech', 'real', 'post'...",They came not from the corporate blog but from...,How Facebook’s new election rules sidestep the...,How Facebook’s new election rules sidestep the...
174,The Interface is a daily column and newsletter...,What Facebook should do about its Kenosha problem,2020-09-01,"['content', 'page', 'guard', 'problem', 'kenos...","The afternoon before the murders, a 3,000-memb...",What Facebook should do about its Kenosha prob...,What Facebook should do about its Kenosha prob...
183,We’re now 61 days away from the U.S. president...,Facebook to block new political ads 1 week bef...,2020-09-03,"['nov', 'voting', 'information', 'elections', ...",CEO Mark Zuckerberg today announced a series o...,Facebook to block new political ads 1 week bef...,Facebook to block new political ads 1 week bef...
258,Facebook is planning to expand its dedicated n...,Facebook plans to expand its news tab beyond t...,2020-08-25,"['content', 'video', 'possible', 'expand', 'ta...",Facebook is planning to expand its dedicated n...,Facebook plans to expand its news tab beyond t...,Facebook plans to expand its news tab beyond t...
261,After restricting a group critical of Thailand...,After restricting a group critical of Thailand...,2020-08-24,"['access', 'critical', 'restrict', 'thailand',...",After restricting a group critical of Thailand...,After restricting a group critical of Thailand...,After restricting a group critical of Thailand...
